In [1]:
# Install uv (if not already installed)
!pip install uv

# Install all dependencies with uv
!uv pip install pandas numpy plotly ipywidgets ipyfilechooser cmocean
# Check and fix widget versions
!uv pip install ipywidgets==8.1.1 jupyterlab-widgets==3.0.9 -q

# You may need to restart kernel after this

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 132.0 MB/s  0:00:00
error: No virtual environment found; run `uv venv` to create an environment, or pass `--system` to install into a non-virtual environment
error: No virtual environment found; run `uv venv` to create an environment, or pass `--system` to install into a non-virtual environment


In [2]:
"""
Trawling4PACE Explorer - v2.1
==============================
Features:
- Reactive updates with loading indicators
- Controls disabled during rendering (prevents race conditions)
- Cancel button for long operations
- Multiple data layers with individual transparency
- CMOcean and Plotly colorscales
- Linear/Log scale options (fixed bug)
- Smart defaults on file load
- Auto bounding box calculation

Author: Leandro (USP/IEAPM)
Project: 2026-proj-Trawling4PACE / NASA PACE Hackweek 2026
"""

from __future__ import annotations

import os
import re
import threading
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

# Try to import cmocean for oceanographic colorscales
try:
    import cmocean
    HAS_CMOCEAN = True
except ImportError:
    HAS_CMOCEAN = False
    print("[WARNING] cmocean not installed. Run: pip install cmocean")


# ============================================================
# COLORSCALE DEFINITIONS
# ============================================================

# Plotly built-in colorscales
PLOTLY_COLORSCALES = [
    "Viridis", "Plasma", "Inferno", "Magma", "Cividis",
    "Turbo", "Hot", "Jet", "Rainbow",
    "Blues", "Greens", "Reds", "Oranges", "Purples",
    "YlOrRd", "YlGnBu", "RdBu", "RdYlBu", "RdYlGn",
    "Picnic", "Portland", "Earth", "Electric", "Blackbody"
]

# CMOcean colorscales (if available)
CMOCEAN_COLORSCALES = [
    "thermal", "haline", "solar", "ice", "gray", "oxy", "deep",
    "dense", "algae", "matter", "turbid", "speed", "amp", "tempo",
    "rain", "phase", "topo", "balance", "delta", "curl", "diff", "tarn"
] if HAS_CMOCEAN else []


def get_cmocean_colorscale(name: str) -> List[List]:
    """
    Convert cmocean colormap to Plotly colorscale format.
    
    Parameters
    ----------
    name : str
        Name of the cmocean colormap
        
    Returns
    -------
    list
        Plotly-compatible colorscale as list of [position, color] pairs
    """
    if not HAS_CMOCEAN:
        return "Viridis"
    
    try:
        cmap = getattr(cmocean.cm, name)
        # Sample colormap at 256 points
        colors = cmap(np.linspace(0, 1, 256))
        # Convert to Plotly format
        colorscale = []
        for i, c in enumerate(colors):
            pos = i / (len(colors) - 1)
            rgb = f"rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})"
            colorscale.append([pos, rgb])
        return colorscale
    except Exception:
        return "Viridis"


def get_all_colorscales() -> Dict[str, List[str]]:
    """
    Get all available colorscales grouped by category.
    
    Returns
    -------
    dict
        Dictionary with 'plotly' and 'cmocean' keys containing colorscale names
    """
    scales = {
        "Plotly": PLOTLY_COLORSCALES,
        "CMOcean": CMOCEAN_COLORSCALES
    }
    return scales


# ============================================================
# PLOTLY API COMPATIBILITY
# ============================================================

def _get_plotly_version() -> tuple:
    """Get Plotly version as tuple for comparison."""
    try:
        version_str = plotly.__version__
        parts = version_str.split('.')
        return tuple(int(p) for p in parts[:3])
    except Exception:
        return (0, 0, 0)


def _have_new_map_api() -> bool:
    """
    Check if new MapLibre API is available (Plotly >= 5.24).
    The new API uses go.Scattermap and layout key 'map' instead of 
    go.Scattermapbox and 'mapbox'.
    """
    version = _get_plotly_version()
    # Plotly 5.24+ uses new MapLibre API
    if version >= (5, 24, 0):
        return hasattr(go, "Scattermap")
    return False


# Detect API once at import - prioritize new API to avoid deprecation warnings
_USE_NEW_API = _have_new_map_api()


def make_scatter_map(**kwargs):
    """
    Create scatter map trace compatible with current Plotly version.
    Uses Scattermap (new) or Scattermapbox (legacy) based on version.
    """
    if _USE_NEW_API:
        return go.Scattermap(**kwargs)
    else:
        return go.Scattermapbox(**kwargs)


def make_density_map(**kwargs):
    """
    Create density map trace compatible with current Plotly version.
    Uses Densitymap (new) or Densitymapbox (legacy) based on version.
    """
    if _USE_NEW_API:
        return go.Densitymap(**kwargs)
    else:
        return go.Densitymapbox(**kwargs)


def apply_map_layout(fig: go.Figure, style: str, center_lat: float, 
                     center_lon: float, zoom: float):
    """
    Apply map layout compatible with current Plotly version.
    Uses 'map' key (new) or 'mapbox' key (legacy) based on version.
    """
    safe_styles = ["open-street-map", "carto-positron", "carto-darkmatter"]
    if style not in safe_styles:
        style = "open-street-map"
    
    layout_map = dict(
        style=style,
        center=dict(lat=center_lat, lon=center_lon),
        zoom=zoom
    )
    
    if _USE_NEW_API:
        fig.update_layout(map=layout_map)
    else:
        fig.update_layout(mapbox=layout_map)


# ============================================================
# COLUMN DETECTION UTILITIES
# ============================================================

def _norm(s: str) -> str:
    """Normalize string for comparison."""
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())


def guess_lat_lon_columns(cols: List[str]) -> Tuple[Optional[str], Optional[str]]:
    """Auto-detect latitude and longitude columns."""
    ncols = {c: _norm(c) for c in cols}
    lat_candidates = [c for c in cols if "lat" in ncols[c]]
    lon_candidates = [c for c in cols if "lon" in ncols[c]]

    def score(c: str) -> int:
        nc = ncols[c]
        s = 0
        if "decdeg" in nc:
            s += 50
        if "beg" in nc or "begin" in nc:
            s += 20
        if "end" in nc:
            s += 5
        return s

    lat = sorted(lat_candidates, key=score, reverse=True)[0] if lat_candidates else None
    lon = sorted(lon_candidates, key=score, reverse=True)[0] if lon_candidates else None
    return lat, lon


def guess_depth_column(cols: List[str]) -> Optional[str]:
    """Auto-detect depth column."""
    prefs = ["AVGDEPTH", "SETDEPTH", "ENDDEPTH", "MAXDEPTH", "MINDEPTH"]
    for p in prefs:
        if p in cols:
            return p
    for c in cols:
        if "depth" in _norm(c):
            return c
    return None


def guess_species_column(cols: List[str]) -> Optional[str]:
    """Auto-detect species column."""
    if "SCIENTIFIC_NAME" in cols:
        return "SCIENTIFIC_NAME"
    for c in cols:
        nc = _norm(c)
        if "scientific" in nc and "name" in nc:
            return c
    return None


def guess_year_month_columns(cols: List[str]) -> Tuple[Optional[str], Optional[str]]:
    """Auto-detect year and month columns."""
    year = None
    month = None
    for c in cols:
        nc = _norm(c)
        if nc in ("estyear", "year"):
            year = c
        if nc in ("estmonth", "month"):
            month = c
    return year, month


def calculate_bounding_box(df: pd.DataFrame, lat_col: str, lon_col: str, 
                           margin: float = 2.0) -> Tuple[float, float, float]:
    """
    Calculate center and zoom level from data extent.
    
    Parameters
    ----------
    df : pd.DataFrame
        Data with coordinates
    lat_col : str
        Latitude column name
    lon_col : str
        Longitude column name
    margin : float
        Margin in degrees to add to extent
        
    Returns
    -------
    tuple
        (center_lat, center_lon, zoom)
    """
    lat = pd.to_numeric(df[lat_col], errors="coerce")
    lon = pd.to_numeric(df[lon_col], errors="coerce")
    
    mask = lat.notna() & lon.notna()
    if mask.sum() == 0:
        return 38.0, -73.0, 5.0
    
    lat_min = lat[mask].min() - margin
    lat_max = lat[mask].max() + margin
    lon_min = lon[mask].min() - margin
    lon_max = lon[mask].max() + margin
    
    center_lat = (lat_min + lat_max) / 2
    center_lon = (lon_min + lon_max) / 2
    
    # Calculate zoom based on extent
    lat_range = lat_max - lat_min
    lon_range = lon_max - lon_min
    max_range = max(lat_range, lon_range)
    
    # Approximate zoom calculation
    if max_range > 40:
        zoom = 3.0
    elif max_range > 20:
        zoom = 4.0
    elif max_range > 10:
        zoom = 5.0
    elif max_range > 5:
        zoom = 6.0
    elif max_range > 2:
        zoom = 7.0
    else:
        zoom = 8.0
    
    return center_lat, center_lon, zoom


# ============================================================
# DATA LAYER CONFIGURATION
# ============================================================

@dataclass
class LayerConfig:
    """
    Configuration for a single data layer.
    
    Attributes
    ----------
    name : str
        Display name for the layer
    variable : str
        Column name for the variable to display
    colorscale : str
        Colorscale name (Plotly or CMOcean)
    colorscale_type : str
        'plotly' or 'cmocean'
    opacity : float
        Layer opacity (0-1)
    scale_type : str
        'linear' or 'log'
    vmin : float, optional
        Minimum value for color scale
    vmax : float, optional
        Maximum value for color scale
    visible : bool
        Whether layer is visible
    marker_size : int
        Marker size for scatter layers
    layer_type : str
        'scatter' or 'density'
    """
    name: str = "Layer"
    variable: str = ""
    colorscale: str = "Viridis"
    colorscale_type: str = "plotly"
    opacity: float = 0.8
    scale_type: str = "linear"
    vmin: Optional[float] = None
    vmax: Optional[float] = None
    visible: bool = True
    marker_size: int = 8
    layer_type: str = "scatter"


# ============================================================
# LAYER WIDGET PANEL
# ============================================================

class LayerPanel:
    """
    Widget panel for configuring a single data layer.
    
    This panel allows users to configure variable, colorscale,
    opacity, scale type, and other layer properties.
    """
    
    def __init__(self, layer_id: int, numeric_cols: List[str], 
                 on_change_callback=None):
        """
        Initialize layer panel.
        
        Parameters
        ----------
        layer_id : int
            Unique identifier for this layer
        numeric_cols : list
            List of numeric column names available
        on_change_callback : callable, optional
            Function to call when any setting changes
        """
        self.layer_id = layer_id
        self.on_change = on_change_callback
        self._building = True  # Flag to prevent callbacks during init
        
        # Build colorscale options
        colorscale_opts = []
        for cs in PLOTLY_COLORSCALES:
            colorscale_opts.append((f"📊 {cs}", f"plotly:{cs}"))
        for cs in CMOCEAN_COLORSCALES:
            colorscale_opts.append((f"🌊 {cs}", f"cmocean:{cs}"))
        
        # Create widgets
        self.w_enabled = widgets.Checkbox(
            value=True,
            description="",
            layout=widgets.Layout(width="30px"),
            indent=False
        )
        
        self.w_variable = widgets.Dropdown(
            options=["(none)"] + numeric_cols,
            value="(none)",
            description="",
            layout=widgets.Layout(width="180px")
        )
        
        self.w_colorscale = widgets.Dropdown(
            options=colorscale_opts,
            value="plotly:Viridis",
            description="",
            layout=widgets.Layout(width="160px")
        )
        
        self.w_opacity = widgets.FloatSlider(
            value=0.8,
            min=0.1,
            max=1.0,
            step=0.05,
            description="",
            readout_format=".2f",
            layout=widgets.Layout(width="140px"),
            continuous_update=False  # Only trigger on release
        )
        
        # FIX: Use ToggleButtons instead of Dropdown for scale type
        # This prevents the bug where selection doesn't change properly
        self.w_scale = widgets.ToggleButtons(
            options=[("Lin", "linear"), ("Log", "log")],
            value="linear",
            description="",
            layout=widgets.Layout(width="100px"),
            style={'button_width': '45px'}
        )
        
        self.w_size = widgets.IntSlider(
            value=8,
            min=3,
            max=20,
            description="",
            layout=widgets.Layout(width="120px"),
            continuous_update=False  # Only trigger on release
        )
        
        self.w_type = widgets.ToggleButtons(
            options=[("Pts", "scatter"), ("Den", "density")],
            value="scatter",
            description="",
            layout=widgets.Layout(width="100px"),
            style={'button_width': '45px'}
        )
        
        # Wire up callbacks for reactive updates
        self._wire_callbacks()
        self._building = False
        
    def _wire_callbacks(self):
        """Connect all widget callbacks for reactive updates."""
        widgets_to_watch = [
            self.w_enabled, self.w_variable, self.w_colorscale,
            self.w_opacity, self.w_scale, self.w_size, self.w_type
        ]
        for w in widgets_to_watch:
            w.observe(self._on_widget_change, names="value")
    
    def _on_widget_change(self, change):
        """Handle widget value change."""
        if not self._building and self.on_change:
            self.on_change()
    
    def set_enabled(self, enabled: bool):
        """Enable or disable all widgets in this panel."""
        self.w_enabled.disabled = not enabled
        self.w_variable.disabled = not enabled
        self.w_colorscale.disabled = not enabled
        self.w_opacity.disabled = not enabled
        self.w_scale.disabled = not enabled
        self.w_size.disabled = not enabled
        self.w_type.disabled = not enabled
    
    def get_config(self) -> Optional[LayerConfig]:
        """
        Get current layer configuration.
        
        Returns
        -------
        LayerConfig or None
            Configuration object, or None if layer is disabled
        """
        if not self.w_enabled.value or self.w_variable.value == "(none)":
            return None
        
        # Parse colorscale
        cs_value = self.w_colorscale.value
        if ":" in cs_value:
            cs_type, cs_name = cs_value.split(":", 1)
        else:
            cs_type, cs_name = "plotly", cs_value
        
        return LayerConfig(
            name=f"Layer {self.layer_id + 1}: {self.w_variable.value}",
            variable=self.w_variable.value,
            colorscale=cs_name,
            colorscale_type=cs_type,
            opacity=self.w_opacity.value,
            scale_type=self.w_scale.value,
            visible=self.w_enabled.value,
            marker_size=self.w_size.value,
            layer_type=self.w_type.value
        )
    
    def update_columns(self, numeric_cols: List[str]):
        """Update available column options."""
        current = self.w_variable.value
        self._building = True
        self.w_variable.options = ["(none)"] + numeric_cols
        if current in numeric_cols:
            self.w_variable.value = current
        else:
            self.w_variable.value = "(none)"
        self._building = False
    
    def set_defaults(self, variable: str = None, colorscale: str = None,
                     scale_type: str = None, layer_type: str = None,
                     opacity: float = None):
        """
        Set default values for this layer.
        
        Parameters
        ----------
        variable : str, optional
            Variable column name
        colorscale : str, optional
            Colorscale in format 'type:name' (e.g., 'cmocean:thermal')
        scale_type : str, optional
            'linear' or 'log'
        layer_type : str, optional
            'scatter' or 'density'
        opacity : float, optional
            Opacity value 0-1
        """
        self._building = True
        
        if variable and variable in [opt for opt in self.w_variable.options]:
            self.w_variable.value = variable
        
        if colorscale:
            # Check if colorscale exists in options
            for opt_label, opt_value in self.w_colorscale.options:
                if opt_value == colorscale:
                    self.w_colorscale.value = colorscale
                    break
        
        if scale_type and scale_type in ["linear", "log"]:
            self.w_scale.value = scale_type
        
        if layer_type and layer_type in ["scatter", "density"]:
            self.w_type.value = layer_type
        
        if opacity is not None:
            self.w_opacity.value = float(opacity)
        
        self._building = False
    
    def get_widget(self) -> widgets.HBox:
        """
        Get the widget container.
        
        Returns
        -------
        widgets.HBox
            Horizontal box containing all layer controls
        """
        return widgets.HBox([
            self.w_enabled,
            widgets.HTML(f"<b>L{self.layer_id + 1}</b>", 
                        layout=widgets.Layout(width="25px")),
            self.w_variable,
            self.w_colorscale,
            self.w_opacity,
            self.w_scale,
            self.w_size,
            self.w_type
        ], layout=widgets.Layout(margin="2px 0"))


# ============================================================
# MAIN DASHBOARD APPLICATION
# ============================================================

class T4PExplorer:
    """
    Trawling4PACE Interactive Explorer.
    
    A reactive dashboard for exploring bottom trawl survey data
    with support for multiple data layers, various colorscales,
    and real-time updates.
    
    Features
    --------
    - Reactive updates with loading indicator
    - Controls disabled during rendering
    - Cancel button for long operations
    - Multiple data layers with individual settings
    - CMOcean and Plotly colorscales
    - Linear and logarithmic color scales
    - Auto-detection of lat/lon columns
    - Smart default layers on file load
    - Auto bounding box calculation
    """

    MAX_LAYERS = 4  # Maximum number of data layers
    MAX_POINTS = 80000  # Maximum points for performance

    def __init__(self, start_dir: Optional[str] = None):
        """
        Initialize the explorer.
        
        Parameters
        ----------
        start_dir : str, optional
            Starting directory for file browser
        """
        self.start_dir = start_dir or os.getcwd()
        if not os.path.isdir(self.start_dir):
            self.start_dir = os.getcwd()

        # Data storage
        self.df: Optional[pd.DataFrame] = None
        self.current_file: Optional[str] = None

        # Column detection cache
        self._species_col: Optional[str] = None
        self._year_col: Optional[str] = None
        self._month_col: Optional[str] = None

        # Layer panels
        self.layer_panels: List[LayerPanel] = []

        # Rendering control
        self._updating = False  # Prevent recursive updates
        self._rendering = False  # Track if currently rendering
        self._cancel_requested = False  # Cancel flag

        # Output widgets
        self.out_fig = widgets.Output(layout={
            "border": "1px solid #ccc",
            "min_height": "550px",
            "width": "100%"
        })
        self.out_log = widgets.Output(layout={
            "border": "1px solid #eee",
            "padding": "6px",
            "max_height": "150px",
            "overflow": "auto"
        })

        # Build UI
        self._build_ui()
        self._wire_callbacks()
        
        # Initial log
        with self.out_log:
            print("=" * 50)
            print("TRAWLING4PACE EXPLORER v2.1")
            print("=" * 50)
            print(f"Plotly version: {plotly.__version__}")
            print(f"CMOcean available: {HAS_CMOCEAN}")
            print(f"Map API: {'MapLibre (new)' if _USE_NEW_API else 'Mapbox (legacy)'}")
            print("=" * 50)

    def _build_ui(self):
        """Build the user interface."""
        
        # === LOADING INDICATOR ===
        self.loading_indicator = widgets.HTML(
            value="",
            layout=widgets.Layout(width="300px")
        )
        
        self.btn_cancel = widgets.Button(
            description="Cancel",
            icon="stop",
            button_style="danger",
            layout=widgets.Layout(width="100px", display="none")
        )
        
        self.progress_bar = widgets.FloatProgress(
            value=0,
            min=0,
            max=100,
            description="",
            bar_style="info",
            layout=widgets.Layout(width="200px", display="none")
        )
        
        # === FILE SELECTION ===
        self.fc = FileChooser(self.start_dir)
        self.fc.filter_pattern = "*.csv"
        self.fc.title = "Select a CSV file"
        self.fc.use_dir_icons = True

        self.btn_load = widgets.Button(
            description="Load CSV",
            icon="upload",
            button_style="success",
            layout=widgets.Layout(width="120px")
        )
        
        self.btn_autozoom = widgets.Button(
            description="Auto-zoom",
            icon="crosshairs",
            layout=widgets.Layout(width="110px")
        )
        
        self.btn_refresh = widgets.Button(
            description="Refresh",
            icon="refresh",
            button_style="primary",
            layout=widgets.Layout(width="100px")
        )

        self.status = widgets.HTML(
            "<b>Status:</b> Select a CSV file and click Load"
        )

        # === COLUMN MAPPING ===
        self.lat_dd = widgets.Dropdown(
            description="Latitude:",
            options=[],
            layout=widgets.Layout(width="320px")
        )
        self.lon_dd = widgets.Dropdown(
            description="Longitude:",
            options=[],
            layout=widgets.Layout(width="320px")
        )

        # === FILTERS ===
        self.species_dd = widgets.Dropdown(
            description="Species:",
            options=["(all)"],
            layout=widgets.Layout(width="450px")
        )
        self.year_ms = widgets.SelectMultiple(
            description="Year:",
            options=[],
            layout=widgets.Layout(width="180px", height="90px")
        )
        self.month_ms = widgets.SelectMultiple(
            description="Month:",
            options=list(range(1, 13)),
            layout=widgets.Layout(width="180px", height="90px")
        )
        self.depth_dd = widgets.Dropdown(
            description="Depth Col:",
            options=["(none)"],
            layout=widgets.Layout(width="280px")
        )
        self.depth_rs = widgets.FloatRangeSlider(
            description="Depth (m):",
            min=0.0,
            max=1000.0,
            value=(0.0, 1000.0),
            continuous_update=False,
            layout=widgets.Layout(width="450px")
        )

        # === MAP SETTINGS ===
        self.basemap_dd = widgets.Dropdown(
            description="Basemap:",
            options=[
                ("OpenStreetMap", "open-street-map"),
                ("Carto Positron (Light)", "carto-positron"),
                ("Carto Dark", "carto-darkmatter")
            ],
            value="open-street-map",  # Default to OpenStreetMap
            layout=widgets.Layout(width="280px")
        )
        self.zoom_slider = widgets.FloatSlider(
            description="Zoom:",
            min=2,
            max=12,
            step=0.5,
            value=5.0,
            continuous_update=False,  # Only trigger on release
            layout=widgets.Layout(width="280px")
        )
        self.center_lat = widgets.FloatText(
            description="Center Lat:",
            value=38.0,
            layout=widgets.Layout(width="160px")
        )
        self.center_lon = widgets.FloatText(
            description="Center Lon:",
            value=-73.0,
            layout=widgets.Layout(width="160px")
        )

        # === LAYER CONFIGURATION ===
        self.layer_container = widgets.VBox([])
        
        # Layer header
        layer_header = widgets.HBox([
            widgets.HTML("<b style='width:30px'>On</b>", layout=widgets.Layout(width="30px")),
            widgets.HTML("<b>#</b>", layout=widgets.Layout(width="25px")),
            widgets.HTML("<b>Variable</b>", layout=widgets.Layout(width="180px")),
            widgets.HTML("<b>Colorscale</b>", layout=widgets.Layout(width="160px")),
            widgets.HTML("<b>Opacity</b>", layout=widgets.Layout(width="140px")),
            widgets.HTML("<b>Scale</b>", layout=widgets.Layout(width="100px")),
            widgets.HTML("<b>Size</b>", layout=widgets.Layout(width="120px")),
            widgets.HTML("<b>Type</b>", layout=widgets.Layout(width="100px")),
        ], layout=widgets.Layout(margin="5px 0", padding="5px", 
                                  background_color="#f0f0f0"))

        # === TABS LAYOUT ===
        tab_data = widgets.VBox([
            widgets.HTML("<h4>📁 File Selection</h4>"),
            self.fc,
            widgets.HBox([self.btn_load, self.btn_refresh, self.btn_autozoom]),
            self.status,
            widgets.HTML("<hr>"),
            widgets.HTML("<h4>🗂️ Coordinate Columns</h4>"),
            widgets.HBox([self.lat_dd, self.lon_dd])
        ])

        tab_filter = widgets.VBox([
            widgets.HTML("<h4>🔍 Data Filters</h4>"),
            self.species_dd,
            widgets.HBox([self.year_ms, self.month_ms]),
            widgets.HTML("<hr>"),
            self.depth_dd,
            self.depth_rs
        ])

        tab_layers = widgets.VBox([
            widgets.HTML("<h4>📊 Data Layers</h4>"),
            widgets.HTML("""
            <p style='color:#666; font-size:12px;'>
            Configure up to 4 layers. Changes apply automatically.
            🌊 = CMOcean colorscale, 📊 = Plotly colorscale
            </p>
            """),
            layer_header,
            self.layer_container
        ])

        tab_map = widgets.VBox([
            widgets.HTML("<h4>🗺️ Map Settings</h4>"),
            self.basemap_dd,
            self.zoom_slider,
            widgets.HBox([self.center_lat, self.center_lon]),
        ])

        self.tabs = widgets.Tab(children=[tab_data, tab_filter, tab_layers, tab_map])
        self.tabs.set_title(0, "📁 Data")
        self.tabs.set_title(1, "🔍 Filters")
        self.tabs.set_title(2, "📊 Layers")
        self.tabs.set_title(3, "🗺️ Map")

        # === STATUS BAR ===
        self.status_bar = widgets.HBox([
            self.loading_indicator,
            self.progress_bar,
            self.btn_cancel
        ], layout=widgets.Layout(margin="5px 0", justify_content="flex-start"))

        # === MAIN LAYOUT ===
        header_html = widgets.HTML("""
        <div style="background: linear-gradient(135deg, #0d3b66 0%, #1a5f7a 100%);
                    padding: 15px; border-radius: 8px; margin-bottom: 10px;">
            <h2 style="color: white; margin: 0; font-family: Arial, sans-serif;">
                🌊 Trawling4PACE Explorer v2.1
            </h2>
            <p style="color: #a8d5e5; margin: 5px 0 0 0; font-size: 13px;">
                Bottom Trawl Survey Explorer | PACE Satellite Integration | 
                <span style="color:#90EE90">Reactive Mode ⚡</span>
            </p>
        </div>
        """)

        self.main_layout = widgets.VBox([
            header_html,
            self.tabs,
            self.status_bar,
            widgets.HTML("<h4>🗺️ Map Output</h4>"),
            self.out_fig,
            widgets.HTML("<h4>📋 Log</h4>"),
            self.out_log
        ])

    def _wire_callbacks(self):
        """Connect all widget callbacks for reactive behavior."""
        
        # Button callbacks
        self.btn_load.on_click(lambda _: self.load_file())
        self.btn_autozoom.on_click(lambda _: self.auto_zoom())
        self.btn_refresh.on_click(lambda _: self.render())
        self.btn_cancel.on_click(lambda _: self._request_cancel())

        # Reactive widgets - these trigger immediate re-render
        reactive_widgets = [
            self.lat_dd, self.lon_dd,
            self.species_dd, self.year_ms, self.month_ms,
            self.depth_dd, self.depth_rs,
            self.basemap_dd, self.zoom_slider,
            self.center_lat, self.center_lon
        ]
        
        for w in reactive_widgets:
            w.observe(self._on_reactive_change, names="value")

    def _on_reactive_change(self, change):
        """Handle reactive widget changes."""
        if not self._updating and not self._rendering and self.df is not None:
            self.render()

    def _request_cancel(self):
        """Request cancellation of current operation."""
        self._cancel_requested = True
        self.loading_indicator.value = "<span style='color:orange'>⏳ Cancelling...</span>"

    def _set_controls_enabled(self, enabled: bool):
        """Enable or disable all controls."""
        # Main buttons
        self.btn_load.disabled = not enabled
        self.btn_autozoom.disabled = not enabled
        self.btn_refresh.disabled = not enabled
        
        # Dropdowns and sliders
        self.lat_dd.disabled = not enabled
        self.lon_dd.disabled = not enabled
        self.species_dd.disabled = not enabled
        self.year_ms.disabled = not enabled
        self.month_ms.disabled = not enabled
        self.depth_dd.disabled = not enabled
        self.depth_rs.disabled = not enabled
        self.basemap_dd.disabled = not enabled
        self.zoom_slider.disabled = not enabled
        self.center_lat.disabled = not enabled
        self.center_lon.disabled = not enabled
        
        # Layer panels
        for panel in self.layer_panels:
            panel.set_enabled(enabled)
        
        # Show/hide cancel button
        self.btn_cancel.layout.display = "none" if enabled else "inline-block"
        self.progress_bar.layout.display = "none" if enabled else "inline-block"

    def _show_loading(self, message: str = "Processing..."):
        """Show loading indicator."""
        self._rendering = True
        self._cancel_requested = False
        self.loading_indicator.value = f"<span style='color:#0066cc'>⏳ {message}</span>"
        self._set_controls_enabled(False)
        self.progress_bar.value = 0

    def _hide_loading(self):
        """Hide loading indicator."""
        self._rendering = False
        self.loading_indicator.value = ""
        self._set_controls_enabled(True)

    def _update_progress(self, value: float, message: str = None):
        """Update progress bar."""
        self.progress_bar.value = value
        if message:
            self.loading_indicator.value = f"<span style='color:#0066cc'>⏳ {message}</span>"

    def _on_layer_change(self):
        """Handle layer configuration change."""
        if not self._updating and not self._rendering and self.df is not None:
            self.render()

    def _create_layer_panels(self, numeric_cols: List[str]):
        """
        Create layer configuration panels.
        
        Parameters
        ----------
        numeric_cols : list
            List of numeric column names
        """
        self.layer_panels = []
        panel_widgets = []
        
        for i in range(self.MAX_LAYERS):
            panel = LayerPanel(
                layer_id=i,
                numeric_cols=numeric_cols,
                on_change_callback=self._on_layer_change
            )
            self.layer_panels.append(panel)
            panel_widgets.append(panel.get_widget())
        
        self.layer_container.children = panel_widgets

    def _set_smart_defaults(self, numeric_cols: List[str]):
        """
        Set smart default values for layers based on available columns.
        
        Layer 1: Surface temperature (SURFTEMP) - linear, thermal, density
        Layer 2: Expected catch weight (EXPCATCHWT) - log, haline, scatter
        """
        if len(self.layer_panels) < 2:
            return
        
        # Layer 1: Temperature
        temp_cols = ["SURFTEMP", "BOTTEMP", "BKTTEMP"]
        temp_col = None
        for col in temp_cols:
            if col in numeric_cols:
                temp_col = col
                break
        
        if temp_col:
            self.layer_panels[0].set_defaults(
                variable=temp_col,
                colorscale="cmocean:thermal",
                scale_type="linear",
                layer_type="density",
                opacity=0.6
            )
        
        # Layer 2: Catch weight
        catch_cols = ["EXPCATCHWT", "EXPCATCHNUM"]
        catch_col = None
        for col in catch_cols:
            if col in numeric_cols:
                catch_col = col
                break
        
        if catch_col:
            self.layer_panels[1].set_defaults(
                variable=catch_col,
                colorscale="cmocean:haline",
                scale_type="log",
                layer_type="scatter",
                opacity=0.8
            )

    def load_file(self):
        """Load selected CSV file."""
        path = self.fc.selected
        if not path or not str(path).lower().endswith(".csv"):
            self.status.value = "<b>Status:</b> ⚠️ Please select a .csv file"
            return

        try:
            self._show_loading("Loading file...")
            self._updating = True
            
            with self.out_log:
                print(f"\n[LOAD] Loading: {path}")

            self._update_progress(20, "Reading CSV...")
            df = pd.read_csv(path, low_memory=False)

            # Check for cancellation
            if self._cancel_requested:
                self._hide_loading()
                self._updating = False
                return

            # Remove index column if present
            if df.columns.size > 0 and (df.columns[0] == "" or 
                                         str(df.columns[0]).startswith("Unnamed")):
                df = df.drop(columns=[df.columns[0]])

            self.df = df
            self.current_file = path

            self._update_progress(40, "Detecting columns...")
            
            # Update column widgets
            self._update_column_widgets()
            
            self._update_progress(60, "Setting up filters...")
            self._update_filter_widgets()

            # Create layer panels
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            self._create_layer_panels(numeric_cols)
            
            self._update_progress(80, "Setting defaults...")
            
            # Set smart defaults
            self._set_smart_defaults(numeric_cols)
            
            # Calculate and set bounding box
            lat_col = self.lat_dd.value
            lon_col = self.lon_dd.value
            if lat_col and lon_col:
                center_lat, center_lon, zoom = calculate_bounding_box(
                    df, lat_col, lon_col, margin=2.0
                )
                self.center_lat.value = round(center_lat, 2)
                self.center_lon.value = round(center_lon, 2)
                self.zoom_slider.value = zoom

            self.status.value = (
                f"<b>Status:</b> ✅ Loaded <code>{os.path.basename(path)}</code> "
                f"({len(df):,} rows, {df.shape[1]} cols)"
            )

            with self.out_log:
                print(f"[OK] {len(df):,} rows loaded")
                print(f"[OK] Numeric columns: {len(numeric_cols)}")
                print(f"[OK] Lat: {self.lat_dd.value}, Lon: {self.lon_dd.value}")
                print(f"[OK] Bounding box: Lat={center_lat:.2f}, Lon={center_lon:.2f}, Zoom={zoom}")

            self._updating = False
            self._update_progress(100, "Rendering...")
            self._hide_loading()
            self.render()

        except Exception as e:
            self._hide_loading()
            self._updating = False
            self.status.value = f"<b>Status:</b> ❌ Error: {str(e)[:50]}"
            with self.out_log:
                print(f"[ERROR] {e}")
                import traceback
                traceback.print_exc()

    def _update_column_widgets(self):
        """Update column selection dropdowns."""
        if self.df is None:
            return

        cols = list(self.df.columns)
        lat_guess, lon_guess = guess_lat_lon_columns(cols)
        depth_guess = guess_depth_column(cols)
        species_guess = guess_species_column(cols)
        year_guess, month_guess = guess_year_month_columns(cols)

        # Update lat/lon dropdowns
        self.lat_dd.options = cols
        self.lon_dd.options = cols

        if lat_guess:
            self.lat_dd.value = lat_guess
        if lon_guess:
            self.lon_dd.value = lon_guess

        # Update depth dropdown
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        self.depth_dd.options = ["(none)"] + numeric_cols
        if depth_guess and depth_guess in numeric_cols:
            self.depth_dd.value = depth_guess

        # Store column guesses
        self._species_col = species_guess
        self._year_col = year_guess
        self._month_col = month_guess

    def _update_filter_widgets(self):
        """Update filter widgets based on data."""
        if self.df is None:
            return

        df = self.df

        # Species filter
        if self._species_col and self._species_col in df.columns:
            species_list = sorted(df[self._species_col].dropna().astype(str).unique().tolist())
            self.species_dd.options = ["(all)"] + species_list
        else:
            self.species_dd.options = ["(all)"]

        # Year filter
        if self._year_col and self._year_col in df.columns:
            years = pd.to_numeric(df[self._year_col], errors="coerce").dropna().astype(int)
            year_opts = sorted(years.unique().tolist())
            self.year_ms.options = year_opts
            self.year_ms.value = tuple(year_opts)

        # Month filter
        self.month_ms.value = tuple(range(1, 13))

        # Depth range
        depth_col = self.depth_dd.value
        if depth_col and depth_col != "(none)" and depth_col in df.columns:
            d = pd.to_numeric(df[depth_col], errors="coerce").dropna()
            if len(d) > 0:
                lo, hi = float(d.min()), float(d.max())
                if np.isfinite(lo) and np.isfinite(hi) and hi > lo:
                    self.depth_rs.min = np.floor(lo)
                    self.depth_rs.max = np.ceil(hi)
                    self.depth_rs.value = (self.depth_rs.min, self.depth_rs.max)

    def _get_filtered_data(self) -> pd.DataFrame:
        """
        Apply all filters and return filtered DataFrame.
        
        Returns
        -------
        pd.DataFrame
            Filtered data
        """
        if self.df is None:
            return pd.DataFrame()

        df = self.df
        lat_col = self.lat_dd.value
        lon_col = self.lon_dd.value

        if lat_col not in df.columns or lon_col not in df.columns:
            return pd.DataFrame()

        # Base filter: valid coordinates
        out = df[df[lat_col].notna() & df[lon_col].notna()].copy()

        # Species filter
        if (self._species_col and self._species_col in out.columns and 
            self.species_dd.value != "(all)"):
            out = out[out[self._species_col].astype(str) == str(self.species_dd.value)]

        # Year filter
        if (self._year_col and self._year_col in out.columns and 
            len(self.year_ms.value) > 0):
            y = pd.to_numeric(out[self._year_col], errors="coerce").astype("Int64")
            out = out[y.isin(list(self.year_ms.value))]

        # Month filter
        if (self._month_col and self._month_col in out.columns and 
            len(self.month_ms.value) > 0):
            m = pd.to_numeric(out[self._month_col], errors="coerce").astype("Int64")
            out = out[m.isin(list(self.month_ms.value))]

        # Depth filter
        depth_col = self.depth_dd.value
        if depth_col and depth_col != "(none)" and depth_col in out.columns:
            d = pd.to_numeric(out[depth_col], errors="coerce")
            lo, hi = self.depth_rs.value
            out = out[(d >= lo) & (d <= hi)]

        # Performance limit
        if len(out) > self.MAX_POINTS:
            out = out.sample(self.MAX_POINTS, random_state=42)

        return out

    def _get_colorscale(self, config: LayerConfig):
        """
        Get Plotly-compatible colorscale from config.
        
        Parameters
        ----------
        config : LayerConfig
            Layer configuration
            
        Returns
        -------
        str or list
            Colorscale specification
        """
        if config.colorscale_type == "cmocean":
            return get_cmocean_colorscale(config.colorscale)
        return config.colorscale

    def _build_figure(self, df: pd.DataFrame) -> go.Figure:
        """
        Build Plotly figure with all layers.
        
        Parameters
        ----------
        df : pd.DataFrame
            Filtered data
            
        Returns
        -------
        go.Figure
            Complete figure
        """
        lat_col = self.lat_dd.value
        lon_col = self.lon_dd.value
        
        fig = go.Figure()
        colorbar_x = 1.02
        
        # Process each layer
        for i, panel in enumerate(self.layer_panels):
            # Check for cancellation
            if self._cancel_requested:
                return fig
            
            config = panel.get_config()
            if config is None:
                continue
                
            if config.variable not in df.columns:
                continue
            
            # Get values
            z = pd.to_numeric(df[config.variable], errors="coerce")
            mask = z.notna()
            
            if mask.sum() == 0:
                continue
            
            # Apply log scale if requested
            z_display = z.copy()
            if config.scale_type == "log":
                # Add small value to avoid log(0)
                z_display = np.log10(z_display.clip(lower=1e-10))
            
            # Get colorscale
            colorscale = self._get_colorscale(config)
            
            # Build trace
            if config.layer_type == "density":
                trace = make_density_map(
                    name=config.name,
                    lat=df.loc[mask, lat_col],
                    lon=df.loc[mask, lon_col],
                    z=z_display.loc[mask],
                    radius=config.marker_size + 5,
                    opacity=config.opacity,
                    colorscale=colorscale,
                    showscale=True,
                    colorbar=dict(
                        title=f"{config.variable}<br>({'log' if config.scale_type == 'log' else 'lin'})",
                        x=colorbar_x,
                        len=0.4,
                        y=0.8 - (i * 0.25)
                    ),
                )
            else:  # scatter
                # Build hover text
                hover_text = df.loc[mask].apply(
                    lambda r: f"<b>{config.variable}</b>: {r[config.variable]:.3f}<br>"
                              f"Lat: {r[lat_col]:.4f}, Lon: {r[lon_col]:.4f}",
                    axis=1
                )
                
                trace = make_scatter_map(
                    name=config.name,
                    mode="markers",
                    lat=df.loc[mask, lat_col],
                    lon=df.loc[mask, lon_col],
                    marker=dict(
                        size=config.marker_size,
                        opacity=config.opacity,
                        color=z_display.loc[mask],
                        colorscale=colorscale,
                        showscale=True,
                        colorbar=dict(
                            title=f"{config.variable}<br>({'log' if config.scale_type == 'log' else 'lin'})",
                            x=colorbar_x,
                            len=0.4,
                            y=0.8 - (i * 0.25)
                        ),
                    ),
                    text=hover_text,
                    hoverinfo="text",
                )
            
            if trace is not None:
                fig.add_trace(trace)
                colorbar_x += 0.08
        
        # Layout
        fig.update_layout(
            height=600,
            margin=dict(l=0, r=80, t=50, b=0),
            title=dict(
                text=f"Trawling4PACE | {os.path.basename(self.current_file) if self.current_file else 'N/A'} | N={len(df):,}",
                x=0.5,
                font=dict(size=14)
            ),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01,
                bgcolor="rgba(255,255,255,0.8)"
            ),
        )
        
        # Apply map layout
        apply_map_layout(
            fig,
            self.basemap_dd.value,
            float(self.center_lat.value),
            float(self.center_lon.value),
            float(self.zoom_slider.value)
        )
        
        return fig

    def render(self):
        """Render the map with current settings."""
        if self._rendering:
            return  # Prevent concurrent renders
        
        with self.out_fig:
            clear_output(wait=True)

            if self.df is None:
                display(widgets.HTML("""
                <div style="padding: 60px; text-align: center; color: #666;">
                    <h3>📂 Load a CSV file to visualize</h3>
                    <p>1. Select a file in the "Data" tab</p>
                    <p>2. Click "Load CSV"</p>
                    <p>3. Configure layers in the "Layers" tab</p>
                </div>
                """))
                return

            try:
                self._show_loading("Filtering data...")
                
                df_f = self._get_filtered_data()

                if self._cancel_requested:
                    self._hide_loading()
                    display(widgets.HTML("<p>Operation cancelled.</p>"))
                    return

                if len(df_f) == 0:
                    self._hide_loading()
                    display(widgets.HTML("""
                    <div style="padding: 40px; text-align: center; color: #c00;">
                        <h3>⚠️ No data after filtering</h3>
                        <p>Adjust filters in the "Filters" tab.</p>
                    </div>
                    """))
                    return

                # Check if any layers are configured
                active_layers = [p.get_config() for p in self.layer_panels 
                                if p.get_config() is not None]
                
                if not active_layers:
                    self._hide_loading()
                    display(widgets.HTML("""
                    <div style="padding: 40px; text-align: center; color: #666;">
                        <h3>📊 No layers configured</h3>
                        <p>Go to the "Layers" tab and select a variable for at least one layer.</p>
                    </div>
                    """))
                    return

                self._update_progress(50, f"Building map ({len(df_f):,} points)...")
                
                fig = self._build_figure(df_f)
                
                if self._cancel_requested:
                    self._hide_loading()
                    display(widgets.HTML("<p>Operation cancelled.</p>"))
                    return
                
                self._update_progress(90, "Rendering...")
                fig.show()
                self._hide_loading()

                with self.out_log:
                    print(f"[RENDER] OK - {len(df_f):,} points, {len(active_layers)} layers")

            except Exception as e:
                self._hide_loading()
                display(widgets.HTML(f"""
                <div style="padding: 20px; color: #c00;">
                    <h3>❌ Render Error</h3>
                    <pre>{str(e)}</pre>
                </div>
                """))
                with self.out_log:
                    print(f"[RENDER ERROR] {e}")
                    import traceback
                    traceback.print_exc()

    def auto_zoom(self):
        """Automatically center and zoom to data extent."""
        if self.df is None:
            return

        self._updating = True
        
        df_f = self._get_filtered_data()
        if len(df_f) == 0:
            self._updating = False
            return

        lat_col = self.lat_dd.value
        lon_col = self.lon_dd.value
        
        center_lat, center_lon, zoom = calculate_bounding_box(
            df_f, lat_col, lon_col, margin=2.0
        )
        
        self.center_lat.value = round(center_lat, 2)
        self.center_lon.value = round(center_lon, 2)
        self.zoom_slider.value = zoom
        
        self._updating = False
        self.render()

    def show(self):
        """Display the explorer."""
        display(self.main_layout)


# ============================================================
# FACTORY FUNCTION & ALIASES
# ============================================================

def create_explorer(start_dir: Optional[str] = None) -> T4PExplorer:
    """
    Create and return a new T4PExplorer instance.
    
    Parameters
    ----------
    start_dir : str, optional
        Starting directory for file browser
        
    Returns
    -------
    T4PExplorer
        Explorer instance
    """
    return T4PExplorer(start_dir=start_dir)


# Alias for backward compatibility
T4PApp = T4PExplorer
create_app = create_explorer


# ============================================================
# AUTO-RUN
# ============================================================

if __name__ == "__main__":
    app = create_explorer()
    app.show()